# Physics-Informed Neural Networks (PINNs) with Fourier Feature Mapping

This notebook implements a Physics-Informed Neural Network (PINN) to solve partial differential equations (PDEs) using Fourier feature mapping. The notebook uses PyTorch for defining and training the network.

```python

$$\frac {\partial u} {\partial t} +u* \frac {\partial u} {\partial x} + v*\frac {\partial u} {\partial y}- \mu *(\frac {\partial^2 u} {\partial x^2}  +{\partial^2 u} {\partial y^2})   = 0$$

$$\frac {\partial v} {\partial t} +u* \frac {\partial v} {\partial x} + v*\frac {\partial v} {\partial y}- \mu *(\frac {\partial^2 v} {\partial x^2}  +{\partial^2 v} {\partial y^2})   = 0$$

# Import necessary libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import grad
import torch.functional as F
import numpy as np
import matplotlib.pyplot as plt


# Define the Fourier Feature Mapping Class


In [2]:
class FFM(nn.Module):
    def __init__(self, in_dim, out_dim, std_dev=2):
        """
        Initializes the Fourier Feature Mapping.
        
        Parameters:
        in_dim (int): Input dimensions (number of independent variables)
        out_dim (int): Output dimensions (length of the hidden layer)
        std_dev (float): Standard deviation for initializing omega parameter
        """
        super().__init__()
        self.omega = nn.Parameter(torch.randn(out_dim, in_dim) * std_dev)

    def forward(self, x):
        """
        Forward pass through the Fourier feature mapping layer.
        
        Parameters:
        x (Tensor): Input tensor
        
        Returns:
        Tensor: Output tensor after applying cosine function
        """
        return torch.cos(F.F.linear(x, self.omega))


# Define the PINNs Network Class

In [3]:
class PINNsNet(nn.Module):
    def __init__(self, in_dim=3, HL_dim=32, out_dim=2, activation=nn.Tanh()):
        """
        Initializes the PINNs network.
        
        Parameters:
        in_dim (int): Input dimensions (number of independent variables)
        HL_dim (int): Width of the network (number of hidden layer units)
        out_dim (int): Output dimensions (number of dependent variables)
        activation (nn.Module): Activation function to use in the network
        """
        super().__init__()
        
        # Define the network architecture
        network = [nn.Linear(in_dim, HL_dim), activation,
                   nn.Linear(HL_dim, HL_dim), activation,
                   nn.Linear(HL_dim, HL_dim), activation,
                   nn.Linear(HL_dim, HL_dim), activation,
                   nn.Linear(HL_dim, out_dim)]
        
        # Define the network using the sequential method
        self.u = nn.Sequential(*network)
        self.v = nn.Sequential(*network)
    
    def forward(self, r,theta, t):
        """
        Forward pass through the network.
        
        Parameters:
        x (Tensor): Spatial coordinates
        y (Tensor): Spatial coordinates
        t (Tensor): Temporal coordinates
        
        Returns:
        Tensor: Network output
        """
        return self.u(torch.cat((r,theta, t), 1)) , self.v(torch.cat((r,theta, t), 1))
    
    def compute_loss(self, r,theta, t, Nr,Ntheta, Nt):
        """
        Computes the loss for the network.
        
        Parameters:
        x (Tensor): Spatial coordinates with gradient tracking enabled
        t (Tensor): Temporal coordinates with gradient tracking enabled
        Nx (int): Number of spatial points
        Nt (int): Number of temporal points
        
        Returns:
        tuple: Losses for PDE, boundary conditions, and initial conditions
        """
        miu=0.1
        r.requires_grad = True
        theta.requires_grad = True
        t.requires_grad = True
        x = r*torch.cos(theta)
        y=r*torch.sin(theta)

        u = self.u(torch.cat((x,y, t), 1))
        v = self.v(torch.cat((x,y, t), 1))
        # Compute PDE derivatives using auto-grad
        u_t = grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_y = grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_yy = grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
        
        v_t = grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_x = grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_y = grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_xx = grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
        v_yy = grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]

        u_r=grad(u, r, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_theta=grad(u, theta, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        v_r=grad(v, r, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_theta=grad(v, theta, grad_outputs=torch.ones_like(v), create_graph=True)[0]

        # Define a loss function
        loss_fun = nn.MSELoss()

        # Compute the PDE residual loss
        resx = u_t + u*u_x + v*u_y - miu * (u_xx + u_yy)
        resy = v_t + u*v_x + v*v_y - miu * (v_xx + v_yy)
        pde_x_loss = loss_fun(resx, torch.zeros_like(resx))
        pde_y_loss = loss_fun(resy, torch.zeros_like(resy))
        # Compute the boundary condition (BC) loss
        u_reshape = u.view(Nr,Ntheta,Nt)
        v_reshape = v.view(Nr,Ntheta,Nt)
        u_theta_reshape = u_theta.view(Nr,Ntheta,Nt)
        v_theta_reshape = v_theta.view(Nr,Ntheta,Nt)
        u_r_reshape = u_theta.view(Nr,Ntheta,Nt)
        v_r_reshape = v_theta.view(Nr,Ntheta,Nt)

        bc_loss= (loss_fun(u_reshape[r==R ,:,:], torch.zeros_like(u_reshape[0,:, :])) +
                   loss_fun(v_reshape[r==R ,:,:], torch.zeros_like(v_reshape[0,:, :])) + 
                   loss_fun(u_theta_reshape[r==R ,:,:], torch.zeros_like(u_theta_reshape[0,:, :])) +
                   loss_fun(v_theta_reshape[r==R ,:,:], torch.zeros_like(v_theta_reshape[0,:, :])) +
                   loss_fun(u_r_reshape[r==R ,:,:], torch.zeros_like(u_r_reshape[0,:, :])) +
                   loss_fun(v_r_reshape[r==R ,:,:], torch.zeros_like(v_r_reshape[0,:, :])) +
                   loss_fun(v_r_reshape[Nr-1 ,:,:], torch.zeros_like(v_reshape[Nr-1,:, :]))+
                   loss_fun(u_reshape[Nr-1 ,0,:], u_reshape[Nr-1 ,:,:]))
        
        

        # Compute the initial condition (IC) loss
        x_reshaped = x.view(Nr, Ntheta,Nt)
        u_initial = 10*u.view(Nr,Ntheta,Nt)[:,:, 0]
        ic_loss = loss_fun(u_initial, u.view(Nr,Ntheta,Nt)[:,:, 0])
    
        return pde_loss, bc_loss, ic_loss


### Define Model and Optimizer

In [4]:
model = PINNsNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Number of points in the domain
Nx, Ny, Nt = 20, 20, 20
Ntheta , Nr= 20, 20
R=0.1
Lr_initial, Lr_final = R, (2**0.5)
Ltheta_initial, Ltheta_final = 0, 2*np.pi
t_initial, t_final = 0, 2
dr = (Lr_final - Lr_initial) / (Nr - 1)
dtheta = (Ltheta_final - Ltheta_initial) / (Ntheta - 1)
dt = (t_final - t_initial) / (Nt - 1)

# Initialize input parameters as tensors
r = torch.zeros(Nr,Ntheta, Nt)
theta = torch.zeros(Nr,Ntheta, Nt)
t = torch.zeros(Nr,Ntheta, Nt)
for i in range(Nr):
    for j in range(Ntheta):
        for k in range(Nt):
            r[i, j, k] = Lr_initial + dr * i
            theta[i, j, k] = Ltheta_initial + dtheta * j
            t[i, j, k] = t_initial + dt * k



In [6]:
# Training loop
for epoch in range(10):
    # Compute various losses
    eq_loss, BC_loss, IC_loss = model.compute_loss(r.view(-1, 1),theta.view(-1, 1), t.view(-1, 1),Nr, Ntheta, Nt)
    # Compute total loss
    total_loss = eq_loss + 20 * BC_loss + 20 * IC_loss

    # Backward pass
    total_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Print epoch and loss
    print(f"Epoch: {epoch}, Loss: {total_loss.item()}")


RuntimeError: shape '[20, 20, 20]' is invalid for input of size 16000